# Random Forest Classifier with pipeline and hyper paramter Tuning

In [1]:
import seaborn as sns
df=sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [5]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [6]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
encoder = LabelEncoder()

In [11]:
df['time']=encoder.fit_transform(df['time'])

In [12]:
df['time']

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [15]:
df['time'].unique()

array([0, 1])

> Dinner -0, Lunch - 1 alphabetically

In [17]:
## independent and dependent feature
X= df.drop(labels=['time'],axis=1)
y=df['time']

In [18]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [19]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
X_train.shape , X_test.shape 

((195, 6), (49, 6))

In [23]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


### Automate FE  in production we use pipelines

like encode smoker , day 

In [24]:
X['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # imputer helps handle missing values using different strategies
from sklearn.preprocessing import StandardScaler ## Feature scaling
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer # tranform columns

In [26]:
categorical_cols=['sex','smoker','day']
numerical_cols=['total_bill','tip','size']

#### Feature engineering automation

In [28]:
# numerical cols handle
num_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), # missing values
        ('scaler',StandardScaler()) # feature scaling
    ]
)

# categorical pipeline
cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')), # missing values
        ('onehotencoder',OneHotEncoder())  # categorical features to numerical
    ]
)

In [29]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [30]:
X_train= preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

## automate model training

In [40]:
from sklearn.ensemble import RandomForestClassifier

models={
    'Random Forest':RandomForestClassifier(),
}

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report ={}
    for i in range(len(models)):
        model= list(models.values())[i]
        # Train Model
        
        model.fit(X_train,y_train)
        
        # predict testing data
        y_test_pred=model.predict(X_test)
        # get r2 scores for test data
        test_model_score=accuracy_score(y_test,y_test_pred)
        
        report[list(models.keys())[i]]=test_model_score
        
    return report

In [43]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877}

In [ ]:
from sklearn.linear_model import LogisticRegression
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic regression':LogisticRegression()
}

evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877, 'Logistic regression': 1.0}

In [45]:
classifier = RandomForestClassifier() # used this because logistic regression has overfitting

params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

from sklearn.model_selection import RandomizedSearchCV

cv= RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [46]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.5s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=1.000 total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.949 total time=   0.5s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.974 total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.923 total time=   0.6s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=300;, score=1.000 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.949 total time=   0.4s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=300;, score=0.923 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [48]:
cv.best_params_

{'n_estimators': 100, 'max_depth': 10, 'criterion': 'gini'}